In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install -q -U git+https://github.com/sbrugman/SDGym.git@v0.2.2-hw

In [ ]:
!pip install -q -U ../../

In [2]:
from timeit import default_timer as timer
from functools import partial
from random import choices
import logging

In [3]:
import sdgym
from sdgym import load_dataset
from sdgym import benchmark
from sdgym import load_dataset

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

In [5]:
import pgmpy

from pgmpy.models import BayesianModel
from pgmpy.estimators import TreeSearch, HillClimbSearch, BicScore, ExhaustiveSearch, BayesianEstimator
from pgmpy.sampling import BayesianModelSampling

In [6]:
import xgboost as xgb
from xgboost import XGBClassifier

In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.isotonic import IsotonicRegression

In [8]:
from scipy import interpolate

In [9]:
from synthsonic.models.kde_utils import kde_smooth_peaks_1dim, kde_smooth_peaks
from synthsonic.models.kde_copula_nn_pdf import KDECopulaNNPdf

In [10]:
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
logging.basicConfig(level=logging.INFO)

In [12]:
dataset_name = 'intrusion_numeric'

In [13]:
data, categorical_columns, ordinal_columns = load_dataset(dataset_name)

#model = CopulaGAN()
#model._MODEL_KWARGS['epochs'] = 1
#model.fit(data, categorical_columns, ordinal_columns)
#distributions = model.get_distributions()

INFO:sdgym.data:Loading dataset intrusion (numeric variables, -1 distinct values threshold)
INFO:sdgym.data:Columns to zero-code: []


In [14]:
categorical_columns, ordinal_columns

([], [])

In [15]:
for i in range(data.shape[1]):
    print (i, len(np.unique(data[:, i])))

0 2123
1 3069
2 9745
3 21
4 18
5 17
6 13
7 484
8 456
9 90
10 47
11 73
12 49
13 98
14 75
15 63
16 256
17 256
18 101
19 101
20 101
21 65
22 97
23 70
24 101
25 101
26 5


# run sdgym

In [16]:
def KDECopulaNNPdf_Synthesizer(real_data, categorical_columns, ordinal_columns):
    # assert len(categorical_columns + ordinal_columns) <= 1

    all_features = list(range(real_data.shape[1]))
    numerical_features = list(set(all_features) - set(categorical_columns + ordinal_columns))
    data = np.float64(real_data)
    
    n_samples = data.shape[0]
    n_features = data.shape[1]
    
    clf = xgb.XGBClassifier(
        n_estimators=250,
        reg_lambda=1,
        gamma=0,
        max_depth=9
    )
#     clf = MLPClassifier(alpha=0.1, random_state=0, max_iter=1000, early_stopping=True)
    
    kde = KDECopulaNNPdf(
        use_KDE=False, 
        categorical_columns=categorical_columns+ordinal_columns+[26],
        distinct_threshold=-1,
        n_bins=25,
        # clf=clf,
        # ordering='mi',
    )
    kde = kde.fit(data)
    
#     X_gen, sample_weight = kde.sample(n_samples)
    X_gen = kde.sample_no_weights(n_samples, show_progress=True, mode='cheap')
    
    X_gen[:, categorical_columns+ordinal_columns] = np.round(X_gen[:, categorical_columns+ordinal_columns+[26]])
    X_gen = np.float32(X_gen)
    
    print(X_gen[:, :5])

    return X_gen

In [17]:
def KDECopulaNNPdf_SynthesizerInteger(real_data, categorical_columns, ordinal_columns):
    """Census has integer only..."""
    data = KDECopulaNNPdf_Synthesizer(real_data, categorical_columns, ordinal_columns)
    data = np.round(data)
    
    print(data[:, :5])

    return data

In [18]:
from sdgym.synthesizers import (
    CLBNSynthesizer, CTGANSynthesizer, IdentitySynthesizer, IndependentSynthesizer,
    MedganSynthesizer, PrivBNSynthesizer, TableganSynthesizer, TVAESynthesizer,
    UniformSynthesizer, VEEGANSynthesizer, CopulaGAN)

all_synthesizers = [
    IdentitySynthesizer,
#     IndependentSynthesizer,
#     KDECopulaNNPdf_Synthesizer,
#    CopulaGAN
     KDECopulaNNPdf_SynthesizerInteger,
]

In [19]:
scores = sdgym.run(synthesizers=all_synthesizers, datasets=[dataset_name], iterations=1)

INFO:sdgym.benchmark:0%|          | 0/2 [00:00<?, ?it/s]
INFO:sdgym.benchmark:Evaluating IdentitySynthesizer on dataset intrusion_numeric; iteration 0; 491.59 MB
INFO:sdgym.data:Loading dataset intrusion (numeric variables, -1 distinct values threshold)
INFO:sdgym.data:Columns to zero-code: []
INFO:sdgym.benchmark:Running IdentitySynthesizer on dataset intrusion_numeric; iteration 0; 532.76 MB
INFO:sdgym.synthesizers.base:Fitting IdentitySynthesizer
INFO:sdgym.synthesizers.base:Sampling IdentitySynthesizer
INFO:sdgym.benchmark:Scoring IdentitySynthesizer on dataset intrusion_numeric; iteration 0; 577.27 MB
INFO:sdgym.evaluate:Evaluating using multiclass classifier DecisionTreeClassifier
INFO:sdgym.evaluate:Evaluating using multiclass classifier MLPClassifier
/Users/maxbaak/opt/miniconda3/envs/synthsonic/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't c

[[-2.7995053e-01 -5.5234868e+03 -1.1831881e+00  6.9660411e-05
   1.2167361e-02]
 [-3.6038995e-01  2.4728777e+03  2.8069157e+01  9.3069888e-05
   2.2620305e-03]
 [-4.6795496e-01 -2.7228435e+03 -1.6506903e+01 -4.1779018e-05
  -5.1692864e-03]
 ...
 [-1.0070992e+00  2.9632371e+03  2.6757179e+01 -3.2871641e-04
  -1.7518742e-02]
 [-9.0241954e-03 -3.6299484e+02  2.1957058e+01  2.1801848e-04
  -7.8057670e-03]
 [-4.4276166e-01  5.1308423e+03 -8.7454514e+01  4.9695413e-04
   8.5613383e-03]]
[[-0.000e+00 -5.523e+03 -1.000e+00  0.000e+00  0.000e+00]
 [-0.000e+00  2.473e+03  2.800e+01  0.000e+00  0.000e+00]
 [-0.000e+00 -2.723e+03 -1.700e+01 -0.000e+00 -0.000e+00]
 ...
 [-1.000e+00  2.963e+03  2.700e+01 -0.000e+00 -0.000e+00]
 [-0.000e+00 -3.630e+02  2.200e+01  0.000e+00 -0.000e+00]
 [-0.000e+00  5.131e+03 -8.700e+01  0.000e+00  0.000e+00]]


INFO:sdgym.evaluate:Evaluating using multiclass classifier DecisionTreeClassifier
INFO:sdgym.evaluate:Evaluating using multiclass classifier MLPClassifier
/Users/maxbaak/opt/miniconda3/envs/synthsonic/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(
INFO:sdgym.benchmark:Finished KDECopulaNNPdf_SynthesizerInteger on dataset intrusion_numeric; iteration 0; 894.22 MB
INFO:sdgym.benchmark:100%|##########| 2/2 [07:40<00:00, 230.10s/it]


In [20]:
scores

,intrusion_numeric/accuracy,intrusion_numeric/macro_f1,intrusion_numeric/micro_f1,timestamp
VEEGANSynthesizer,NaN,NaN,NaN,2020-04-12 09:41:35.096775
CLBNSynthesizer,NaN,NaN,NaN,2020-10-17 09:46:54.494331
CTGAN,NaN,NaN,NaN,2020-10-17 09:46:54.494331
CTGANSynthesizer,NaN,NaN,NaN,2020-10-17 09:46:54.494331
CopulaGAN,NaN,NaN,NaN,2020-10-17 09:46:54.494331
GaussianCopulaCategorical,NaN,NaN,NaN,2020-10-17 09:46:54.494331
GaussianCopulaCategoricalFuzzy,NaN,NaN,NaN,2020-10-17 09:46:54.494331
GaussianCopulaOneHot,NaN,NaN,NaN,2020-10-17 09:46:54.494331
IndependentSynthesizer,NaN,NaN,NaN,2020-10-17 09:46:54.494331
MedganSynthesizer,NaN,NaN,NaN,2020-10-17 09:46:54.494331


In [21]:
scores.tail(4)

,intrusion_numeric/accuracy,intrusion_numeric/macro_f1,intrusion_numeric/micro_f1,timestamp
TableganSynthesizer,NaN,NaN,NaN,2020-10-17 09:46:54.494331
UniformSynthesizer,NaN,NaN,NaN,2020-10-17 09:46:54.494331
IdentitySynthesizer,0.99888,0.870465,0.99888,2021-05-13 20:38:38.658756
KDECopulaNNPdf_SynthesizerInteger,0.98256,0.579864,0.98256,2021-05-13 20:38:38.658756
